# The Guardian API

In the `beautiful_soup.ipynb` notebook, I showed how BeautifulSoup can be used 
to parse messy HTML, tp extract information, and to act as a rudimentary web crawler. 
I used The Guardian as an illustrative example about how this can be achieved. 
The reason for choosing The Guardian was because they provide a REST API to their servers. 
With theise it is possible to perform specific queries on their servers, and to receive 
current information from their servers according to their API guide (ie in JSON)

http://open-platform.theguardian.com/

In order to use their API, you will need to register for an API key. 
At the time of writing (Feb 1, 2017) this was an automated process that can be completed at 

http://open-platform.theguardian.com/access/

The API is documented here: 

http://open-platform.theguardian.com/documentation/

and Python bindings to their API are provided by The Guardian here

https://github.com/prabhath6/theguardian-api-python

and these can easily be integrated into a web-crawler based on API calls, rather than being based 
on HTML parsing, etc. 

We use four parameters in our queries here: 

1. `section`: the section of the newspaper that we are interested in querying. In this case I'm lookin in 
the technology section 

2. `order-by`: I have specifie that the newest items should be closer to the front of the query list 

3. `api-key`: I have left this as test (which works here), but for *real* deployment of such a spider
a real API key should be specified 

4. `page-size`: The number of results to return. 

In [1]:
from __future__ import print_function

import requests 
import json 

# Inspect all sections and search for technology-based sections

In [2]:
url = 'https://content.guardianapis.com/sections?api-key=test'
req = requests.get(url)
src = req.text 

In [4]:
src

'{"response":{"status":"ok","userTier":"developer","total":73,"results":[{"id":"about","webTitle":"About","webUrl":"https://www.theguardian.com/about","apiUrl":"https://content.guardianapis.com/about","editions":[{"id":"about","webTitle":"About","webUrl":"https://www.theguardian.com/about","apiUrl":"https://content.guardianapis.com/about","code":"default"}]},{"id":"artanddesign","webTitle":"Art and design","webUrl":"https://www.theguardian.com/artanddesign","apiUrl":"https://content.guardianapis.com/artanddesign","editions":[{"id":"artanddesign","webTitle":"Art and design","webUrl":"https://www.theguardian.com/artanddesign","apiUrl":"https://content.guardianapis.com/artanddesign","code":"default"}]},{"id":"australia-news","webTitle":"Australia news","webUrl":"https://www.theguardian.com/australia-news","apiUrl":"https://content.guardianapis.com/australia-news","editions":[{"id":"australia-news","webTitle":"Australia news","webUrl":"https://www.theguardian.com/australia-news","apiUrl":"

In [3]:
sections = json.loads(src)['response']

print(sections.keys())

dict_keys(['status', 'userTier', 'total', 'results'])


In [6]:
print(json.dumps(sections['results'][0], indent=2, sort_keys=True))

{
  "apiUrl": "https://content.guardianapis.com/about",
  "editions": [
    {
      "apiUrl": "https://content.guardianapis.com/about",
      "code": "default",
      "id": "about",
      "webTitle": "About",
      "webUrl": "https://www.theguardian.com/about"
    }
  ],
  "id": "about",
  "webTitle": "About",
  "webUrl": "https://www.theguardian.com/about"
}


In [7]:
for result in sections['results']: 
    if 'tech' in result['id'].lower(): 
        print(result['webTitle'], result['apiUrl'])

Technology https://content.guardianapis.com/technology


# Manual query on whole API

In [9]:
# Specify the arguments
args = {
    'section': 'technology', 
    'order-by': 'newest', 
    'api-key': 'test', 
    'page-size': '100'
}

# Construct the URL
base_url = 'http://content.guardianapis.com/search'
url = '{}?{}'.format(
    base_url, 
    '&'.join(["{}={}".format(kk, vv) for kk, vv in args.items()])
)

# Make the request and extract the source
req = requests.get(url) 
src = req.text

In [10]:
print('Number of byes received:', len(src))

Number of byes received: 60001


The API returns JSON, so we parse this using the in-built JSON library. 
The API specifies that all data are returned within the `response` key, even under failure. 
Thereofre, I have immediately descended to the response field 

# Parsing the JSON

In [12]:
response = json.loads(src)['response']
print('The following are available:\n ', sorted(response.keys()))

The following are available:
  ['currentPage', 'orderBy', 'pageSize', 'pages', 'results', 'startIndex', 'status', 'total', 'userTier']


# Verifying the status code

It is important to verify that the status message is `ok` before continuing - if it is not `ok` no 'real' data 
will have been received. 

In [13]:
assert response['status'] == 'ok'

# Listing the results 

The API standard states that the results will be found in the `results` field under the `response` field. 
Furthermore, the URLs will be found in the `webUrl` field, and the title will be found in the `webTitle` 
field. 

First let's look to see what a single result looks like in full, and then I will print a restricted 
set of parameters on the full set of results .

In [14]:
print(json.dumps(response['results'][0], indent=2, sort_keys=True))

{
  "apiUrl": "https://content.guardianapis.com/technology/2018/jan/24/self-driving-cars-dangerous-period-false-security",
  "id": "technology/2018/jan/24/self-driving-cars-dangerous-period-false-security",
  "isHosted": false,
  "pillarId": "pillar/news",
  "pillarName": "News",
  "sectionId": "technology",
  "sectionName": "Technology",
  "type": "article",
  "webPublicationDate": "2018-01-24T08:01:25Z",
  "webTitle": "Who's driving? Autonomous cars may be entering the most dangerous phase",
  "webUrl": "https://www.theguardian.com/technology/2018/jan/24/self-driving-cars-dangerous-period-false-security"
}


In [15]:
for result in response['results']: 
    print(result['webUrl'][:70], result['webTitle'][:20])

https://www.theguardian.com/technology/2018/jan/24/self-driving-cars-d Who's driving? Auton
https://www.theguardian.com/technology/2018/jan/23/mary-lee-berners-le Mary Lee Berners-Lee
https://www.theguardian.com/technology/2018/jan/23/elon-musk-aiming-fo Elon Musk lines up $
https://www.theguardian.com/technology/2018/jan/23/facebook-new-privac Facebook to roll out
https://www.theguardian.com/technology/2018/jan/23/apple-homepod-avail Apple HomePod finall
https://www.theguardian.com/technology/2018/jan/23/bitcoin-ubs-chairma UBS chairman warns a
https://www.theguardian.com/technology/shortcuts/2018/jan/23/a-sign-th ‘A sign that you’re 
https://www.theguardian.com/technology/2018/jan/23/cybercrime-130bn-st Cybercrime: £130bn s
https://www.theguardian.com/technology/2018/jan/22/cyber-attack-on-uk- Major cyber-attack o
https://www.theguardian.com/technology/2018/jan/22/rupert-murdoch-face Rupert Murdoch tells
https://www.theguardian.com/us-news/2018/jan/22/amazon-go-convenience- Amazon Go